## Notebook to compare final results MELD graph vs MELD per vertex
Results on test cohort and withheld sites H27 H28 H101

In [1]:
#imports
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from meld_graph.training import tp_fp_fn_tn, dice_coeff
from meld_classifier.meld_cohort import MeldSubject,MeldCohort
from meld_graph.icospheres import IcoSpheres
from meld_graph.graph_tools import GraphTools
import itertools
import pandas as pd
from meld_graph.evaluation import load_prediction
from meld_graph import experiment

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/kw350


### load per-vertex results

In [18]:
#On test dataset
# ref = '/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/iteration_21-09-15/ensemble_21-09-15/fold_all/results'

# with h5py.File(os.path.join(ref, 'predictions_ensemble_iteration.hdf5'), "r") as f:
#     subjects = list(f.keys())
# subjects.remove('MELD_H4_3T_FCD_0011') # because does not exist in graph model
# cohort = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5',
#                dataset='MELD_dataset_V6.csv')
# print(len(subjects))


# On wittheld sites
refh27h28 = '/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/predict_NewSiteH27H28_21-09-20/fold_all/results'
refh101 = '/rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/predict_NewSiteH101_22-08-19/results'


with h5py.File(os.path.join(refh27h28, 'predictions_ensemble_iteration.hdf5'), "r") as f:
    subjects = list(f.keys())
    
with h5py.File(os.path.join(refh101, 'predictions_ensemble_iteration.hdf5'), "r") as f:
    subjects = subjects + list(f.keys())
    
#only needed to get the lesion mask and borderzone  
cohort= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6_kernels_robustCombat_NewSite.hdf5', dataset='MELD_dataset_NewSiteH27H28H101.csv')
print(len(subjects))

221


In [37]:
df_old=pd.DataFrame()
subjects_dictionary={}
values={}
for si,subj in enumerate(subjects):
    if si%100==0:
        print(si)
    values['ID']=subj
    s = MeldSubject(subj,cohort=cohort)
    values['group']= True if s.group=='patient' else False
    labels_hemis = {}
    dists={}
    labels = np.zeros(len(cohort.cortex_label)*2)
    for hemi in ['lh','rh']:
        dists[hemi], labels_hemis[hemi] = s.load_feature_lesion_data(
                    features=['.on_lh.boundary_zone.mgh'], hemi=hemi, features_to_ignore=[]
                )
        if np.sum(dists[hemi])==0:
            dists[hemi] +=200
    labels = np.hstack([labels_hemis['lh'][cohort.cortex_mask],labels_hemis['rh'][cohort.cortex_mask]])
    borderzones = np.vstack([dists['lh'][cohort.cortex_mask,:],dists['rh'][cohort.cortex_mask,:]]).ravel()<20
    #load pred from old classifier
    if 'H101' in subj:
        pred_file_old = os.path.join(refh101, 'predictions_ensemble_iteration.hdf5')
    elif ('H27' in subj) or ('H28' in subj):
        pred_file_old = os.path.join(refh27h28, 'predictions_ensemble_iteration.hdf5')
    else:
        pred_file_old = os.path.join(ref, 'predictions_ensemble_iteration.hdf5')

    result_hemis_old = load_prediction(subj,pred_file_old, dset='prediction')
    result_old = np.hstack([result_hemis_old['lh'],result_hemis_old['rh']])
    values['model']='per vertex'
    
    #add detection with borderzone
    if labels.sum()>0:
        values['detected'] = np.logical_and(result_old, borderzones).any()
        values['number TP clusters'] = len(set(result_old[borderzones.astype('bool')]))-1
        values['size_pred'] = len(result_old[borderzones.astype('bool')])
    else:
        values['number TP clusters'] = 0
    # add number of FP clusters : total clusters - TP clusters
    values['number FP clusters']=len(set(result_hemis_old['lh']))+len(set(result_hemis_old['rh']))-2-values['number TP clusters']
    df_old=pd.concat([df_old,pd.DataFrame([values])])
df_old = df_old.reset_index()
df_old.head()

0
100
200


,index,ID,group,model,number TP clusters,number FP clusters,detected,size_pred
0,0,MELD_H27_3T_C_0018,False,per vertex,0,1,NaN,NaN
1,0,MELD_H27_3T_C_0019,False,per vertex,0,1,NaN,NaN
2,0,MELD_H27_3T_C_0020,False,per vertex,0,0,NaN,NaN
3,0,MELD_H27_3T_C_0021,False,per vertex,0,5,NaN,NaN
4,0,MELD_H27_3T_C_0022,False,per vertex,0,0,NaN,NaN


In [38]:
df_old.groupby('group')['ID'].count()

group
False    103
True     118
Name: ID, dtype: int64

In [39]:
print('all together')
dfsub = df_old.copy()
sensitivity = np.mean(dfsub['detected'][dfsub['group']])
specificity = (dfsub['number FP clusters'][dfsub['group']==0]>0).mean()
total_detected = np.sum(dfsub['number TP clusters'][dfsub['group']])
total_predicted = np.sum(dfsub['number FP clusters'][dfsub['group']])
ppv = total_detected / (total_predicted + total_detected)
print(np.round(sensitivity,2),np.round(1-specificity,2),np.round(ppv,2))

try:
    for site in ['H27', 'H28', 'H101']:
        print(site)
        dfsub = df_old[df_old['ID'].str.contains(site)]
        sensitivity = np.mean(dfsub['detected'][dfsub['group']])
        specificity = (dfsub['number FP clusters'][dfsub['group']==0]>0).mean()
        total_detected = np.sum(dfsub['number TP clusters'][dfsub['group']])
        total_predicted = np.sum(dfsub['number FP clusters'][dfsub['group']])
        ppv = total_detected / (total_predicted + total_detected)
        print(np.round(sensitivity,2),np.round(1-specificity,2),np.round(ppv,2))
except:
    pass

all together
0.78 0.59 0.5
H27
0.94 0.17 0.52
H28
0.62 nan 0.32
H101
0.78 0.68 0.56


### load results for meld graph model

In [40]:
# # for test dataset
# experiment_dir = '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350'
# model = '23-10-30_FOPF_dcop'
# # model = '23-10-30_MSBS_dcop_with_combat'
# df_model = pd.read_csv(os.path.join(experiment_dir,model,
#                                           's_0','fold_all','results_best_model','test_results.csv'))

# for withheld sites
experiment_dir = '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350'
model = '23-10-30_FOPF_dcop'
# model = '23-10-30_MSBS_dcop_with_combat'

df_model_h27 = pd.read_csv(os.path.join(experiment_dir,model,'s_0','fold_all', 'test_H27','results_best_model','test_results.csv'))
df_model_h28 = pd.read_csv(os.path.join(experiment_dir,model,'s_0','fold_all', 'test_H28','results_best_model','test_results.csv'))
df_model_h101 = pd.read_csv(os.path.join(experiment_dir,model,'s_0','fold_all', 'test_H101','results_best_model','test_results.csv'))
df_model = pd.concat([df_model_h27, df_model_h28, df_model_h101])

In [15]:
df_model.groupby('group')['ID'].count()

group
False    103
True     118
Name: ID, dtype: int64

In [16]:
df_model['model'] = ['graph' for x in df_model.iterrows()]
df_model = df_model[['ID','group','detected','number FP clusters','number TP clusters', 'model']]
df_model.head()

,ID,group,detected,number FP clusters,number TP clusters,model
0,MELD_H27_3T_FCD_0001,True,False,3,0,graph
1,MELD_H27_3T_FCD_0002,True,True,0,1,graph
2,MELD_H27_3T_FCD_0003,True,True,0,2,graph
3,MELD_H27_3T_FCD_0004,True,True,0,1,graph
4,MELD_H27_3T_FCD_0005,True,False,1,0,graph


In [17]:
print('all together')
dfsub = df_model.copy()
sensitivity = np.mean(dfsub['detected'][dfsub['group']])
specificity = (dfsub['number FP clusters'][dfsub['group']==0]>0).mean()
total_detected = np.sum(dfsub['number TP clusters'][dfsub['group']])
total_predicted = np.sum(dfsub['number FP clusters'][dfsub['group']])
ppv = total_detected / (total_predicted + total_detected)
print(np.round(sensitivity,2),np.round(1-specificity,2),np.round(ppv,2))

try:
    for site in ['H27','H28', 'H101']:
        print(site)
        dfsub = df_model[df_model['ID'].str.contains(site)]
        sensitivity = np.mean(dfsub['detected'][dfsub['group']])
        specificity = (dfsub['number FP clusters'][dfsub['group']==0]>0).mean()
        total_detected = np.sum(dfsub['number TP clusters'][dfsub['group']])
        total_predicted = np.sum(dfsub['number FP clusters'][dfsub['group']])
        ppv = total_detected / (total_predicted + total_detected)
        print(np.round(sensitivity,2),np.round(1-specificity,2),np.round(ppv,2))
except:
    pass

all together
0.66 0.7 0.69
H27
0.82 0.28 0.65
H28
0.69 nan 0.46
H101
0.62 0.79 0.78


In [10]:
# ## Concat results

# df = pd.concat([df_old, df_model], ignore_index=True)
# df.head()

### add breakdown on test dataset

In [43]:
# for test dataset
experiment_dir = '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350'
model = '23-10-30_FOPF_dcop'
df_model = pd.read_csv(os.path.join(experiment_dir,model,
                                          's_0','fold_all','results_best_model','test_results.csv'))

cohort= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6_kernels.hdf5', dataset='MELD_dataset_NewSiteH27H28H101.csv')


In [44]:
# add demographic
age_array=[]
sex_array=[]
histo_array=[]
site_array=[]
scanner_array=[]
flair_array=[]
group_array=[]
sf_array=[]
mri_negative_array=[]
for subject in df_model['ID']:
    subj = MeldSubject(subject, cohort)
    age, sex, histo, site, sf, mri_negative = subj.get_demographic_features(["Age at preoperative", "Sex", "Histology", "Site", "Seizure free", "Ever reported MRI negative"])
    scanner_array.append(subj.scanner)
    age_array.append(age)
    sex_array.append(sex)
    histo_array.append(histo)
    site_array.append(site)
    sf_array.append(sf)
    flair_array.append(subj.has_flair)
    mri_negative_array.append(mri_negative)

    
df_model['Age at preoperative']=age_array
df_model['Sex']=sex_array
df_model['Histology']=histo_array
df_model['Site']=site_array
df_model['Scanner']=scanner_array
df_model['FLAIR']=flair_array
df_model['Seizure free']=sf_array
df_model['Ever reported MRI negative'] = mri_negative_array

df_model.head()

,ID,group,detected,number FP clusters,number TP clusters,tp,fp,fn,tn,dice lesional,dice non-lesional,Age at preoperative,Sex,Histology,Site,Scanner,FLAIR,Seizure free,Ever reported MRI negative
0,MELD_H2_15T_FCD_0001,True,False,0,0,0,0,271,293533,3.690037e-18,0.999539,20.0,1.0,NaN,H2,15T,False,1.0,0.0
1,MELD_H2_15T_FCD_0003,True,True,0,1,2530,4163,1370,285741,4.776740e-01,0.990411,10.0,0.0,NaN,H2,15T,False,NaN,0.0
2,MELD_H2_15T_FCD_0005,True,False,1,0,0,6728,262,286814,1.430615e-19,0.987961,20.0,1.0,NaN,H2,15T,False,NaN,0.0
3,MELD_H2_15T_FCD_0007,True,True,0,1,2062,3241,201,288300,5.450700e-01,0.994066,4.0,1.0,FCD_2B,H2,15T,False,1.0,0.0
4,MELD_H2_15T_FCD_0008,True,False,1,0,0,297,970,292537,7.892660e-19,0.997839,10.0,1.0,NaN,H2,15T,False,NaN,0.0


In [45]:
pat = df_model[df_model['group']==True]

In [46]:
disp_df=pd.DataFrame(100*pat.groupby('Scanner').mean()['detected']).round(1)
disp_df['count'] = pat.groupby('Scanner').count()['detected']
disp_df

,detected,count
Scanner,,
15T,64.3,56
3T,69.6,204


In [23]:
disp_df=pd.DataFrame(100*pat.groupby(['Scanner','FLAIR']).mean()['detected']).round(1)
disp_df['count'] = pat.groupby(['Scanner','FLAIR']).count()['detected']
disp_df

detected  count
Scanner FLAIR                 
15T     False      58.3     36
        True       75.0     20
3T      False      71.1    114
        True       67.8     90

In [24]:
disp_df=pd.DataFrame(100*pat.groupby(['Seizure free']).mean()['detected']).round(1)
disp_df['count'] = pat.groupby(['Seizure free']).count()['detected']
disp_df

,detected,count
Seizure free,,
0.0,66.7,51
1.0,78.3,106


In [50]:
disp_df=pd.DataFrame(100*pat.groupby(['Sex']).mean()['detected']).round(1)
disp_df['count'] = pat.groupby(['Sex']).count()['detected']
disp_df

,detected,count
Sex,,
0.0,63.2,125
1.0,73.3,135


In [57]:
pat['Histology'] = pat['Histology'].fillna('not available')
disp_df=pd.DataFrame(100*pat.groupby(['Histology']).mean()['detected']).round(1)
disp_df['n patients'] = pat.groupby(['Histology']).count()['detected']
disp_df.rename(columns={'detected':'% Detected'})

/tmp/ipykernel_88579/3010077965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pat['Histology'] = pat['Histology'].fillna('not available')


,% Detected,n patients
Histology,,
FCD_1,69.2,13
FCD_2A,73.7,57
FCD_2B,79.6,93
FCD_3,75.0,8
not available,52.8,89


In [54]:
pat['Histology']

0         NaN
1         NaN
2         NaN
3      FCD_2B
4         NaN
        ...  
448       NaN
449       NaN
450    FCD_2B
451    FCD_2A
452     FCD_1
Name: Histology, Length: 260, dtype: object

In [52]:
pat['Histology']

0         NaN
1         NaN
2         NaN
3      FCD_2B
4         NaN
        ...  
448       NaN
449       NaN
450    FCD_2B
451    FCD_2A
452     FCD_1
Name: Histology, Length: 260, dtype: object

In [26]:
disp_df=pd.DataFrame(100*pat.groupby(['Ever reported MRI negative']).mean()['detected']).round(1)
disp_df['count'] = pat.groupby(['Ever reported MRI negative']).count()['detected']
disp_df

,detected,count
Ever reported MRI negative,,
0.0,71.7,180
1.0,61.3,80


In [47]:
pat['paediatric'] = pat['Age at preoperative']<18
disp_df=pd.DataFrame(100*pat.groupby(['paediatric']).mean()['detected']).round(1)
disp_df['count'] = pat.groupby(['paediatric']).count()['detected']
disp_df

/tmp/ipykernel_88579/3746894774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pat['paediatric'] = pat['Age at preoperative']<18


,detected,count
paediatric,,
False,68.7,131
True,68.2,129
